## Installing Beautiful soup for web scrapping

In [1]:
!pip install requests BeautifulSoup4 fire

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


## importing necessary modules

In [2]:

from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import numpy as np
import fire

In [3]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('finding all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)


## Most influential Twitter handles

In [4]:
res1 = get_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa',tag='h2')

## splitting to get Twitter handles from the web scrape

In [8]:
df = pd.DataFrame(res1).head(100)
df
df1 = df[0].str.split('.', expand=True)
df2 = df1[1].str.split('(', expand=True)
df2[1] = df2[1].str.strip(')')
df2.columns = ['name','username']
# print(df2)
handle = df2["username"]
handle1 = pd.DataFrame(handle, columns = ['username'])
handle1[::-1].reset_index(drop=True)

,username
0,@Trevornoah
1,@GarethCliff
2,None
3,@News24
4,@Julius_S_Malema
...,...
95,@JamesCopnall
96,@AfricaCheck
97,@andiMakinana
98,@a24media
